In [1]:
import os
import pandas as pd
import polars as pl
pd.set_option('display.max_rows', 100)
pl.Config.set_tbl_cols(100)
pl.Config.set_tbl_rows(100)

%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path

import polars as pl

from EventStream.data.dataset_polars import Dataset

DATA_DIR = Path("../MIMIC_ESD_new_schema_08-31-23-1")
ESD = Dataset.load(DATA_DIR)

events_df = ESD.events_df.filter(~pl.all_horizontal(pl.all().is_null()))
dynamic_measurements_df = ESD.dynamic_measurements_df.filter(~pl.all_horizontal(pl.all().is_null()))

df_data = (events_df
            .join(dynamic_measurements_df, on="event_id", how="left")
            .drop(['event_id'])
            .sort(by=['subject_id', 'timestamp', 'event_type'])
            )


# df_data = pl.read_csv("sample_data.csv")

/home/justinxu/miniconda3/envs/esgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Updating config.save_dir from /n/data1/hms/dbmi/zaklab/RAMMS/data/MIMIC_IV/ESD_new_schema_08-31-23-1 to ../MIMIC_ESD_new_schema_08-31-23-1
Loading events from ../MIMIC_ESD_new_schema_08-31-23-1/events_df.parquet...
Loading dynamic_measurements from ../MIMIC_ESD_new_schema_08-31-23-1/dynamic_measurements_df.parquet...


# End-to-End

In [30]:
#filter to just 226 id
df_temp = df_data.filter(pl.col("subject_id") == 226)

In [31]:
df_temp = df_temp.to_pandas()

In [34]:
df_temp = df_temp[df_temp['event_type'] == 'LAB']

#cast lab to string
df_temp['lab'] = df_temp['lab'].astype(str)

# find rows that contain substring 'Hemoglobin'
df_temp = df_temp[df_temp['lab'].str.contains('Hemoglobin')]

In [35]:
df_temp

,subject_id,timestamp,event_type,age,time_of_day,measurement_id,admission_type,admission_location,language,race,...,lab,valuenum,totalamountuom,rate,rateuom,infusion,totalamount,patientweight,procedure,medication
2231,226,2126-01-17 14:00:00,LAB,71.043349,PM,37188895.0,NaN,NaN,NaN,NaN,...,Hemoglobin (g/dl),11.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2771,226,2126-01-18 00:05:00,LAB,71.044500,EARLY_AM,109386447.0,NaN,NaN,NaN,NaN,...,Hemoglobin (g/dl),11.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2857,226,2126-01-18 03:21:00,LAB,71.044872,EARLY_AM,109386471.0,NaN,NaN,NaN,NaN,...,Hemoglobin (g/dl),9.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2939,226,2126-01-18 06:15:00,LAB,71.045203,AM,109386492.0,NaN,NaN,NaN,NaN,...,Hemoglobin (g/dl),8.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5082,226,2126-01-19 21:00:00,LAB,71.049624,LATE_PM,73292242.0,NaN,NaN,NaN,NaN,...,Hemoglobin (g/dl),12.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6239,226,2126-01-20 20:00:00,LAB,71.052247,PM,73292521.0,NaN,NaN,NaN,NaN,...,Hemoglobin (g/dl),12.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7556,226,2126-01-21 20:00:00,LAB,71.054985,PM,1090665.0,NaN,NaN,NaN,NaN,...,Hemoglobin (g/dl),13.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9306,226,2126-01-23 16:00:00,LAB,71.060005,PM,1091093.0,NaN,NaN,NaN,NaN,...,Hemoglobin (g/dl),9.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10569,226,2127-11-25 18:30:00,LAB,72.897388,PM,1088029.0,NaN,NaN,NaN,NaN,...,Hemoglobin (g/dl),12.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10758,226,2127-11-25 22:00:00,LAB,72.897787,LATE_PM,73290086.0,NaN,NaN,NaN,NaN,...,Hemoglobin (g/dl),7.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
from task_querying_v2 import *

name = 'abnormal_lab'

config_path = f"../sample_configs/{name}.yaml"

res = query_task(config_path, df_data)
res

Loading config...

Generating predicate columns...

Added predicate column is_lab.
Added predicate column is_hemoglobin.
Added predicate column is_abnormal_value.
Added predicate column is_normal_value.
Added predicate column is_abnormal_hemoglobin.
Added predicate column is_normal_hemoglobin.

Building tree...
trigger
┣━━ gap
┃   ┗━━ target
┗━━ input


12127 subjects (14688370 rows) were excluded due to trigger condition: {'predicate': 'normal_hemoglobin', 'min': 1}.


Querying...


Querying subtree rooted at gap...


Querying subtree rooted at target...


Querying subtree rooted at input...
11 subjects (11 rows) were excluded due to constraint: [(col("is_any")) >= (5)].


Done.



subject_id,trigger/timestamp,gap/timestamp,target/timestamp,input/timestamp,gap/window_summary,target/window_summary,input/window_summary,label
u16,datetime[μs],datetime[μs],datetime[μs],datetime[μs],struct[7],struct[7],struct[7],i32
0,2131-01-14 15:00:00,2131-01-15 15:00:00,2131-01-22 15:00:00,2131-02-13 15:00:00,"{64,64,0,0,1475,2,117}","{222,228,0,2,5952,12,460}","{286,292,0,2,7427,14,577}",0
0,2131-01-16 16:00:00,2131-01-17 16:00:00,2131-01-24 16:00:00,2131-02-15 16:00:00,"{52,52,0,0,1436,1,112}","{116,122,0,1,3056,8,240}","{168,174,0,1,4492,9,352}",0
0,2131-01-17 20:09:00,2131-01-18 20:09:00,2131-01-25 20:09:00,2131-02-16 20:09:00,"{55,59,0,0,1302,3,101}","{52,53,0,0,1600,3,118}","{107,112,0,0,2902,6,219}",0
2,2156-04-25 15:00:00,2156-04-26 15:00:00,2156-05-03 15:00:00,2156-05-25 15:00:00,"{29,29,0,0,790,1,59}","{81,83,0,0,1406,0,144}","{617,632,0,0,13693,13,1146}",0
3,2137-03-01 01:00:00,2137-03-02 01:00:00,2137-03-09 01:00:00,2137-03-31 01:00:00,"{57,59,0,0,1164,3,118}","{286,294,0,0,5875,4,488}","{460,472,0,0,9520,11,848}",0
13,2148-01-07 08:30:00,2148-01-08 08:30:00,2148-01-15 08:30:00,2148-02-06 08:30:00,"{84,88,0,0,1951,3,220}","{13,13,0,0,394,0,48}","{97,101,0,0,2345,3,270}",0
20,2167-12-17 22:01:00,2167-12-18 22:01:00,2167-12-25 22:01:00,2168-01-16 22:01:00,"{24,25,0,0,328,0,55}","{0,0,0,0,0,0,0}","{24,25,0,0,328,0,56}",0
24,2177-03-26 05:00:00,2177-03-27 05:00:00,2177-04-03 05:00:00,2177-04-25 05:00:00,"{73,74,0,1,1504,7,177}","{142,143,0,0,2698,7,310}","{215,217,0,1,4202,14,487}",0
24,2177-03-26 18:06:00,2177-03-27 18:06:00,2177-04-03 18:06:00,2177-04-25 18:06:00,"{56,56,0,0,1212,5,149}","{106,107,0,0,2124,5,222}","{162,163,0,0,3336,10,371}",0


In [4]:
# res.select(['target/window_summary']).unnest('target/window_summary')

# Profiling

In [ ]:
import cProfile, pstats, sys
from task_querying_v2 import *

name = 'profiling'
config_path = f"test_configs/{name}.yaml"

total_rows = [10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000, 5000000, 10000000, 50000000, 100000000, 150000000]

for i in total_rows:
    df_temp = df_data.head(i)
    print(f"Number of rows: {df_temp.shape[0]}")
    print(f"Number of patients: {df_temp['subject_id'].n_unique()}")

    pr = cProfile.Profile()
    pr.enable()

    df_result = query_task(config_path, df_temp, verbose=False)

    pr.disable()
    ps = pstats.Stats(pr, stream=sys.stdout)
    ps.sort_stats("cumtime").print_stats()

    print(f"Number of rows: {df_result.shape[0]}")
    print(f"Number of patients: {df_result['subject_id'].n_unique()}")
    print("=====================================NEXT=====================================")

Number of rows: 10
Number of patients: 1
         175709 function calls (173457 primitive calls) in 0.350 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.350    0.350 /home/justinxu/esgpt/ESGPTTaskQuerying/to_organize/task_querying_v2.py:818(query_task)
     11/1    0.016    0.001    0.267    0.267 /home/justinxu/esgpt/ESGPTTaskQuerying/to_organize/task_querying_v2.py:543(query_subtree)
      179    0.001    0.000    0.217    0.001 /home/justinxu/miniconda3/envs/esgpt/lib/python3.10/site-packages/polars/lazyframe/frame.py:1814(collect)
      179    0.205    0.001    0.205    0.001 {method 'collect' of 'builtins.PyLazyFrame' objects}
       34    0.001    0.000    0.149    0.004 /home/justinxu/miniconda3/envs/esgpt/lib/python3.10/site-packages/polars/dataframe/frame.py:6305(join)
       10    0.018    0.002    0.124    0.012 /home/justinxu/esgpt/ESGPTTaskQuerying/to_organize/task_querying_v

KeyboardInterrupt: 

# Unit Testing

In [ ]:
from simple_test_runner import *

In [ ]:
df_test_1_input = pl.DataFrame(
    {
        'subject_id': [1, 1, 1],
        'timestamp': ['12/1/1900 12:00', '12/1/1900 13:00', '12/1/1900 14:00'],
        'event_type': ['ADMISSION', 'LAB', 'DISCHARGE'],
        'dx': ['', '', ''],
        'lab_test': ['', 'SpO2', ''],
        'lab_value': ['', '99', ''],
        'is_admission': [1, 0, 0],
        'is_lab': [0, 1, 0],
        'is_discharge': [0, 0, 1],
        'is_any': [1, 1, 1],
    }
)

df_test_1_predicate_cols = [col for col in df_test_1_input.columns if col.startswith('is_')]

df_test_1_endpoint_expr = (False, "is_discharge", True, None) 

test_1_anchor_to_subtree_root_by_subtree_anchor = pl.DataFrame(
    {
        'subject_id': [1],
        'timestamp': ['12/1/1900 12:00'],
        'is_admission': [0],
        'is_lab': [0],
        'is_discharge': [0],
        'is_any': [0],
    }
)

df_test_1_result = pl.DataFrame(
    {
        "subject_id": [1],
        "timestamp": ["12/1/1900 14:00"],
        "timestamp_at_anchor": ["12/1/1900 12:00"],
        "is_admission": [0],
        "is_lab": [1],
        "is_discharge": [1],
        "is_any": [2],
        "is_admission_summary": [0],
        "is_lab_summary": [0],
        "is_discharge_summary": [0],
        "is_any_summary": [0],
    }
)

In [ ]:
test_1 = [
    {
        "message": "Testing summarize_event_bound_window, should be equal...",
        "args": (
            df_test_1_input, 
            df_test_1_predicate_cols, 
            df_test_1_endpoint_expr,
            test_1_anchor_to_subtree_root_by_subtree_anchor),
        "want": df_test_1_result,
    }
]

simple_test_runner(test_1, summarize_event_bound_window)

Testing summarize_event_bound_window, should be equal... Passed!


# Integration Testing

### Config Loading

In [ ]:
from task_querying_v2 import *

In [ ]:
data = pl.read_csv('sample_data.csv')
data = data.with_columns(pl.col('timestamp').str.strptime(pl.Datetime, format='%m/%d/%Y %H:%M').cast(pl.Datetime))

In [ ]:
cfg = load_config('sample_config.yaml')
cfg

{'predicates': {'admission': {'column': 'event_type', 'value': 'ADMISSION'},
  'discharge': {'column': 'event_type', 'value': 'DISCHARGE'},
  'death': {'column': 'event_type', 'value': 'DEATH'},
  'covid': {'column': 'dx', 'value': 'COVID'},
  'discharge_or_death': {'type': 'ANY', 'predicates': ['discharge', 'death']}},
 'windows': {'trigger': {'start': 'admission',
   'duration': None,
   'offset': None,
   'end': 'admission',
   'excludes': None,
   'includes': [{'predicate': 'admission', 'min': 1, 'max': 1}],
   'st_inclusive': False,
   'end_inclusive': True},
  'gap': {'start': 'trigger.end',
   'duration': '48 hours',
   'offset': None,
   'end': 'gap.start + gap.duration',
   'excludes': [{'predicate': 'admission'}, {'predicate': 'discharge'}, {'predicate': 'death'}],
   'includes': None,
   'st_inclusive': False,
   'end_inclusive': True},
  'target': {'start': 'gap.end',
   'duration': None,
   'offset': None,
   'end': 'discharge_or_death',
   'excludes': None,
   'includes':

### Generate predicate columns

In [ ]:
predicates_df = generate_predicate_columns(cfg, data)
predicates_df

AttributeError: 'DotAccessibleDict' object has no attribute 'system'

### Build tree

In [ ]:
tree = build_tree_from_config(cfg)
print_tree(tree, style="const_bold")
for each_node in preorder_iter(tree):
    print(each_node)
cfg

trigger
┣━━ gap
┃   ┗━━ target
┗━━ input
Node(/trigger, constraints={'is_admission': (1, 1)}, endpoint_expr=(False, 'is_admission', True, datetime.timedelta(0)))
Node(/trigger/gap, constraints={'is_admission': (None, 0), 'is_discharge': (None, 0), 'is_death': (None, 0)}, endpoint_expr=(False, datetime.timedelta(days=2), True, datetime.timedelta(0)))
Node(/trigger/gap/target, constraints={'is_discharge_or_death': (1, 1)}, endpoint_expr=(True, 'is_discharge_or_death', True, datetime.timedelta(0)))
Node(/trigger/input, constraints={'is_covid': (None, 0), 'is_any': (5, None)}, endpoint_expr=(False, datetime.timedelta(days=30), True, datetime.timedelta(days=1)))


{'predicates': {'admission': {'column': 'event_type', 'value': 'ADMISSION'},
  'discharge': {'column': 'event_type', 'value': 'DISCHARGE'},
  'death': {'column': 'event_type', 'value': 'DEATH'},
  'covid': {'column': 'dx', 'value': 'COVID'},
  'discharge_or_death': {'type': 'ANY', 'predicates': ['discharge', 'death']}},
 'windows': {'trigger': {'start': 'admission',
   'duration': None,
   'offset': None,
   'end': 'admission',
   'excludes': None,
   'includes': [{'predicate': 'admission', 'min': 1, 'max': 1}],
   'st_inclusive': False,
   'end_inclusive': True},
  'gap': {'start': 'trigger.end',
   'duration': '48 hours',
   'offset': None,
   'end': 'gap.start + gap.duration',
   'excludes': [{'predicate': 'admission'}, {'predicate': 'discharge'}, {'predicate': 'death'}],
   'includes': None,
   'st_inclusive': False,
   'end_inclusive': True},
  'target': {'start': 'gap.end',
   'duration': None,
   'offset': None,
   'end': 'discharge_or_death',
   'excludes': None,
   'includes':

### Summarize window

In [ ]:
import datetime

predicate_cols = [col for col in predicates_df.columns if col.startswith("is_")]
print(predicate_cols)

anchor_to_subtree_root_by_subtree_anchor = (
        predicates_df.filter(predicates_df['is_admission'] == 1)
            .select('subject_id', 'timestamp', *[pl.col(c) for c in predicate_cols])
            .with_columns('subject_id', 'timestamp', *[pl.lit(0).alias(c) for c in predicate_cols])
    )
print(anchor_to_subtree_root_by_subtree_anchor)

['is_admission', 'is_discharge', 'is_death', 'is_covid', 'is_discharge_or_death', 'is_any']
shape: (6, 8)
┌────────────┬─────────────┬─────────────┬─────────────┬──────────┬──────────┬────────────┬────────┐
│ subject_id ┆ timestamp   ┆ is_admissio ┆ is_discharg ┆ is_death ┆ is_covid ┆ is_dischar ┆ is_any │
│ ---        ┆ ---         ┆ n           ┆ e           ┆ ---      ┆ ---      ┆ ge_or_deat ┆ ---    │
│ i64        ┆ datetime[μs ┆ ---         ┆ ---         ┆ i32      ┆ i32      ┆ h          ┆ i32    │
│            ┆ ]           ┆ i32         ┆ i32         ┆          ┆          ┆ ---        ┆        │
│            ┆             ┆             ┆             ┆          ┆          ┆ i32        ┆        │
╞════════════╪═════════════╪═════════════╪═════════════╪══════════╪══════════╪════════════╪════════╡
│ 1          ┆ 1989-12-01  ┆ 0           ┆ 0           ┆ 0        ┆ 0        ┆ 0          ┆ 0      │
│            ┆ 12:03:00    ┆             ┆             ┆          ┆          ┆        

Temporally-bound

In [ ]:
summarize_temporal_window(predicates_df=predicates_df, predicate_cols=predicate_cols, endpoint_expr=(False, timedelta(days=2), True, None), anchor_to_subtree_root_by_subtree_anchor=anchor_to_subtree_root_by_subtree_anchor)

subject_id,timestamp,timestamp_at_anchor,is_admission,is_discharge,is_death,is_covid,is_discharge_or_death,is_any,is_admission_summary,is_discharge_summary,is_death_summary,is_covid_summary,is_discharge_or_death_summary,is_any_summary
i64,datetime[μs],datetime[μs],i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
1,1989-12-01 12:03:00,1989-12-01 12:03:00,0,1,0,0,1,7,0,0,0,0,0,0
1,1991-01-27 23:32:00,1991-01-27 23:32:00,0,0,0,0,0,3,0,0,0,0,0,0
1,1991-03-03 19:33:00,1991-03-03 19:33:00,0,0,1,0,1,2,0,0,0,0,0,0
2,1996-03-08 02:24:00,1996-03-08 02:24:00,0,1,0,1,1,4,0,0,0,0,0,0
2,1996-06-05 00:32:00,1996-06-05 00:32:00,0,0,0,0,0,0,0,0,0,0,0,0
3,1996-03-08 02:24:00,1996-03-08 02:24:00,0,0,1,0,1,6,0,0,0,0,0,0


Event-bound

In [ ]:
summarize_event_bound_window(predicates_df=predicates_df, predicate_cols=predicate_cols, endpoint_expr=(False, 'is_discharge_or_death', True, None), anchor_to_subtree_root_by_subtree_anchor=anchor_to_subtree_root_by_subtree_anchor)

subject_id,timestamp,timestamp_at_anchor,is_admission,is_discharge,is_death,is_covid,is_discharge_or_death,is_any,is_admission_summary,is_discharge_summary,is_death_summary,is_covid_summary,is_discharge_or_death_summary,is_any_summary
i64,datetime[μs],datetime[μs],i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
1,1989-12-02 15:00:00,1989-12-01 12:03:00,0,1,0,0,1,7,0,0,0,0,0,0
1,1991-01-31 02:15:00,1991-01-27 23:32:00,0,1,0,0,1,9,0,0,0,0,0,0
1,1991-03-03 21:38:00,1991-03-03 19:33:00,0,0,1,0,1,2,0,0,0,0,0,0
2,1996-03-08 16:00:00,1996-03-08 02:24:00,0,1,0,1,1,4,0,0,0,0,0,0
2,1996-06-08 03:00:00,1996-06-05 00:32:00,0,0,1,0,1,5,0,0,0,0,0,0
3,1996-03-10 00:00:00,1996-03-08 02:24:00,0,0,1,0,1,6,0,0,0,0,0,0
